In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [86]:
!pip install textract

In [87]:
from gensim.models import Word2Vec
import string
import re
import os
import collections
from os import listdir
from os.path import isfile, join
from joblib import dump, load
import pickle
from io import StringIO
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import PhraseMatcher
from itertools import chain
import textract
from gensim.models import Word2Vec
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim.models.phrases import Phraser, Phrases
import nltk
import collections
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)
    resumeText = re.sub('RT|cc', ' ', resumeText)
    resumeText = re.sub('#\S+', '', resumeText)
    resumeText = re.sub('@\S+', '  ', resumeText)
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)
    return resumeText

In [89]:
def Preprocessfile(filename):
  text = textract.process(filename)
  text= text.decode('utf-8').replace("\n", " ")
  # print(text)
  x=[]
  tokens=word_tokenize(text)
  tok=[w.lower() for w in tokens]
  table=str.maketrans('','',string.punctuation)
  strpp=[w.translate(table) for w in tok]
  words=[word for word in strpp if word.isalpha()]
  stop_words=set(stopwords.words('english'))
  words=[w for w in words if not w in stop_words]
  x.append(words)
  # print(x)
  res=" ".join(chain.from_iterable(x))
  return res

In [90]:
def calculate_score(text):
  #print the similarity score
  # print("\n Similarity Score: ")
  cv = CountVectorizer()
  count_matrix = cv.fit_transform(text)
  # print(cosine_similarity(count_matrix))
  matchpercent = cosine_similarity(count_matrix)[0][1]*100
  return round(matchpercent,2)

def print_score(text):
  print("Your Resume matches about " + str(calculate_score(text)) + "% of the job")

#JD file

In [91]:
import os

# Check if the file exists
file_path = "jd.txt"
if os.path.exists(file_path):
    # If the file exists, delete it
    os.remove(file_path)

# Create the file again
with open(file_path, 'w') as file:
    file.write("This is the new content of jd.txt\n")

x=Preprocessfile('jd.txt')

In [92]:
# import os

# # Define the directory containing the PDF files
# directory = 'testing/'

# # Initialize a list to store filenames and their corresponding scores
# files_scores = []

# # Iterate over each file in the directory
# for filename in os.listdir(directory):
#     if filename.endswith('.pdf'):
#         filepath = os.path.join(directory, filename)

#         # Preprocess the current PDF file
#         y = Preprocessfile(filepath)

#         # Assuming 'x' is defined elsewhere or loaded from somewhere
#         text = [y, x]

#         # Compute the score for the current PDF file
#         score = calculate_score(text)  # Assuming compute_score function is defined

#         # Append filename and score as a tuple to the list
#         files_scores.append((filename, score))

# # Sort the list based on the scores in decreasing order
# files_scores.sort(key=lambda x: x[1], reverse=True)

# # Print filenames and scores in decreasing order
# for filename, score in files_scores:
#     print(f"Filename: {filename}, Score: {score}")


#deleting speific folder

In [93]:
import os
import shutil

def delete_folder(folder_path):
    """
    Delete a folder if it exists.

    Parameters:
        folder_path (str): Path to the folder to be deleted.
    """
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' deleted successfully.")
    else:
        print(f"Folder '{folder_path}' does not exist.")

# Example usage:
# folder_to_delete = 'all_bucket_files'

# delete_folder(folder_to_delete)


In [94]:
from google.cloud import storage
import os

def download_bucket_files2(bucket_name, folder_name, output_folder):
    """
    Download PDF and DOCX files from a specified folder in a Google Cloud Storage bucket into a specified folder.

    Parameters:
        bucket_name (str): Name of the Google Cloud Storage bucket.
        folder_name (str): Name of the folder within the bucket from which files will be downloaded.
        output_folder (str): Path to the folder where files will be downloaded.
    """
    # Create a client to interact with the GCS API
    client = storage.Client()

    # Get the bucket
    bucket = client.get_bucket(bucket_name)

    # List all blobs (files) in the specified folder
    blobs = bucket.list_blobs(prefix=folder_name)

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Download PDF, DOCX, and DOC files to the specified folder
    for blob in blobs:
        # Ensure that the blob represents a file (not a subfolder)
        if blob.name.endswith('.pdf') or blob.name.endswith('.docx') or blob.name.endswith('.doc'):
            # Construct the file path
            relative_path = os.path.relpath(blob.name, folder_name)
            file_path = os.path.join(output_folder, relative_path)

            # Download the file to local storage
            blob.download_to_filename(file_path)
            print(f"File '{file_path}' downloaded from folder '{folder_name}' in bucket '{bucket_name}'")

# Example usage
# bucket_name = "hackathon1415"
# folder_name = "RESUME/data"
# output_folder = "path_to_output_folder"

# delete_folder('path_to_output_folder')
# download_bucket_files(bucket_name, folder_name, output_folder)


In [95]:
from google.colab import auth
from google.cloud import storage
import os

def download_bucket_files(bucket_name, output_folder):

    # Authenticate with your Google Cloud account
    auth.authenticate_user()

    # Create a client to interact with the GCS API
    client = storage.Client()

    # Get the bucket
    bucket = client.get_bucket(bucket_name)

    # List all blobs (files) in the bucket
    blobs = bucket.list_blobs()

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Download PDF and DOCX files to the specified folder
    for blob in blobs:
        if blob.name.endswith('.pdf') or blob.name.endswith('.docx') or blob.name.endswith('.doc'):
            # Construct the file path
            file_path = os.path.join(output_folder, os.path.basename(blob.name))
            # Download the file to local storage
            blob.download_to_filename(file_path)
            print(f"File '{file_path}' downloaded from bucket '{bucket_name}'")

# Example usage:
# bucket_name = 'hackathontestdata2024'
# output_folder = 'pdf_docx_files'

# download_bucket_files(bucket_name, output_folder)


#Downloading resumes

In [96]:
# bucket_name = 'hackathontestdata2024'
# output_folder = 'all_bucket_files'
# download_bucket_files(bucket_name, output_folder)

#function to get n no of resumes with their matching values

In [97]:
import os

def process_files_in_directory(directory, n):# n is no of best resumes we want
    # Initialize a list to store filenames and their corresponding scores
    files_scores = []

    # Iterate over each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(('.pdf', '.docx', '.doc')):
            filepath = os.path.join(directory, filename)

            # Preprocess the current PDF file
            y = Preprocessfile(filepath)

            # Assuming 'x' is defined elsewhere or loaded from somewhere
            text = [y, x]

            # Compute the score for the current PDF file
            score = calculate_score(text)  # Assuming compute_score function is defined

            # Append filename and score as a tuple to the list
            files_scores.append((filename, round((score), 2)))

    # Sort the list based on the scores in decreasing order
    files_scores.sort(key=lambda x: x[1], reverse=True)

    files_scores=files_scores[:n]

    converted_data = []

    # Counter for id
    id_counter = 1
    for item in files_scores:
        converted_item = {
            "id": id_counter,
            "score": item[1],
            "path": item[0]
        }
        converted_data.append(converted_item)
        id_counter += 1


    return converted_data

# Example usage:

#Example usage


In [98]:
# # Example usage:
# directory = 'all_bucket_files'
# sorted_files_scores = process_files_in_directory(directory, 3)
# print(sorted_files_scores)

In [99]:
# delete_folder('all_bucket_files')

In [100]:
#function to check if file is presennt, if yes then it will deleted

import os

def check_and_create_file(file_name, text_to_write):
    # Check if the file exists
    if os.path.exists(file_name):
        # If the file exists, delete it
        os.remove(file_name)
        print(f"File '{file_name}' exists and has been deleted.")

    # Create a new file
    with open(file_name, 'w') as f:
        f.write(""+text_to_write)

    print(f"File '{file_name}' has been created.")

# Usage
# file_name = "example.txt"
# check_and_create_file(file_name, 'hi hello')


In [101]:
import os

def check_and_create_folder(folder_name):
    # Check if the folder exists
    if os.path.exists(folder_name):
        # If the folder exists, delete it
        os.rmdir(folder_name)
        # print(f"Folder '{folder_name}' exists and has been deleted.")

    # Create a new folder
    os.mkdir(folder_name)
    # print(f"Folder '{folder_name}' has been created.")

# Usage example
# check_and_create_folder("example_folder")


#download_files_from_bucket_link

In [102]:
#delete_folder('demo2')

#Download data example


In [103]:

# bucket_link = "hackathontestdata2024/"
# destination_folder = "demo2"

# parts = bucket_link.strip('/').split('/')
# first_variable = parts[0]
# other_words = '/'.join(parts[1:])


# delete_folder(destination_folder)
# download_bucket_files(first_variable, other_words, destination_folder)

In [104]:
def extract_bucket_name(url):
    # Split the URL by '/'
    parts = url.split('/')

    # Find the index of 'browser' segment
    browser_index = parts.index('browser')

    # Extract the bucket name and folder path
    bucket_name = parts[browser_index + 1]
    folder_path = '/'.join(parts[browser_index + 2:])

    return f"{bucket_name}/{folder_path}"


# bucket_name = extract_bucket_name("https://console.cloud.google.com/storage/browser/hackathontestdata2024")
# print("Bucket Name:", bucket_name)

#adding Flask API

In [105]:
!pip install pyngrok

In [ ]:
from flask import Flask
from flask import redirect, url_for, request
from pyngrok import ngrok

port_no = 5000

app = Flask(__name__)
ngrok.set_auth_token("2doQclucgVWyOkPuodbpl28hfCd_5yVcWjSuiL461kLtfYYSa")
public_url =  ngrok.connect(port_no).public_url


@app.route("/")
def home():
    return "hi everyone"

# @app.route('/get_resumes_listxx', methods=['POST', 'GET'])
# def my_test_endpoint():
#     # input_json = request.get_json(force=True)
#     if request.method == 'POST':
#         if request.is_json:
#             data = request.get_json()
#             data['city'] = 'Bangalore'
#             data['state'] = 'Karnataka'
#             data['country'] = 'India'
#             return data


@app.route('/get_resumes_list', methods=['POST'])
def my_test_endpoint():
    if request.method == 'POST':
        if request.is_json:
            data = request.get_json()
            # data['city'] = 'Bangalore'
            # data['state'] = 'Karnataka'
            # data['country'] = 'India'
            # return data
            context_str=data['context']
            category_str=data['category']
            threshold_str=data['threshold']
            noOfMatches_str=data['noOfMatches']
            inputPath_str=data['inputPath']

            # data['hello']='not so hello'
            # return data

            response_data = {}

            # take job desc from API call and write in file
            file_name = "job_desc.txt"
            check_and_create_file(file_name, ''+context_str)

            #take input path url from API call and download all files inside the folder
            bucket_path = extract_bucket_name(inputPath_str)
            destination_folder = "resume_folder"
            delete_folder(destination_folder)
            check_and_create_folder(destination_folder)

            parts = bucket_path.strip('/').split('/')
            first_variable = parts[0]
            other_words = '/'.join(parts[1:])
            download_bucket_files2(first_variable, other_words, destination_folder)


            #now searching for resumes
            directory = destination_folder
            sorted_files_scores = process_files_in_directory(directory, (int(noOfMatches_str)))
            # print(sorted_files_scores)

            response_data['results']=sorted_files_scores
            # return response_data

            total_score = 0
            max_value=0
            for item in sorted_files_scores:
                total_score += item['score']
                max_value=max(max_value, item['score'])

            # confidenceScore = ((int(len(response_data['results'])))/(int(noOfMatches_str)))*((total_score)/(int(len(sorted_files_scores))))
            confidenceScore = max_value
            confidenceScore=round((confidenceScore), 2)
            response_data["metadata"] = {"confidenceScore": confidenceScore}
            response_data["count"]=len(sorted_files_scores)
            response_data["status"]='success'

            return response_data



print(f"To acces the Gloable link please click {public_url}")

app.run(port=port_no)

To acces the Gloable link please click https://c8de-34-70-189-202.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


File 'job_desc.txt' exists and has been deleted.
File 'job_desc.txt' has been created.
Folder 'resume_folder' deleted successfully.
File 'resume_folder/10001727.pdf' downloaded from folder '' in bucket 'hackathontestdata2024'
File 'resume_folder/10005171.pdf' downloaded from folder '' in bucket 'hackathontestdata2024'
File 'resume_folder/10030015.pdf' downloaded from folder '' in bucket 'hackathontestdata2024'
File 'resume_folder/10041713.pdf' downloaded from folder '' in bucket 'hackathontestdata2024'
File 'resume_folder/10062724.pdf' downloaded from folder '' in bucket 'hackathontestdata2024'
File 'resume_folder/10070224.pdf' downloaded from folder '' in bucket 'hackathontestdata2024'
File 'resume_folder/3547447.pdf' downloaded from folder '' in bucket 'hackathontestdata2024'


INFO:werkzeug:127.0.0.1 - - [18/Mar/2024 13:49:00] "POST /get_resumes_list HTTP/1.1" 200 -
